# Sandbox
For playing around during development.

In [1]:
%pylab inline
import pysd
#print pysd.__version__
print pysd.__file__
import pandas as pd
import os


Populating the interactive namespace from numpy and matplotlib
pysd/__init__.pyc


/Users/houghton/anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [3]:
model = pysd.read_vensim('./tests/test-models/tests/abs/test_abs.mdl')
model

ValueError: shape mismatch: value array of shape (3,2) could not be broadcast to indexing result of shape (1,2,3)

In [4]:
%debug

> /Users/houghton/anaconda/lib/python2.7/site-packages/xarray/core/indexing.py(362)__setitem__()
    360     def __setitem__(self, key, value):
    361         key = self._convert_key(key)
--> 362         self.array[key] = value
    363 
    364 

ipdb> u
> /Users/houghton/anaconda/lib/python2.7/site-packages/xarray/core/variable.py(378)__setitem__()
    376                             'method or accessing its .values attribute.')
    377         data = orthogonally_indexable(self._data_cached())
--> 378         data[key] = value
    379 
    380     @property

ipdb> u
> /Users/houghton/anaconda/lib/python2.7/site-packages/xarray/core/dataarray.py(416)__setitem__()
    414         else:
    415             # orthogonal array indexing
--> 416             self.variable[key] = value
    417 
    418     def __delitem__(self, key):

ipdb> u
> /Users/houghton/anaconda/lib/python2.7/site-packages/xarray/core/dataarray.py(106)__setitem__()
    104 
    105     def __setitem__(self

In [2]:
pysd.read_vensim('./tests/test-models/tests/subscript_3d_arrays/test_subscript_3d_arrays.mdl')

ValueError: shape mismatch: value array of shape (3,2) could not be broadcast to indexing result of shape (1,2,3)

In [3]:
%debug

> /Users/houghton/Google_Drive/Academic Projects/PYSD/pysd/pysd/pysd.py(254)initialize_state()
    252                     initialization_order.append(key)
    253                 except KeyError:  # may also need to catch TypeError?
--> 254                     retry_flag = True
    255             if not making_progress:
    256                 raise KeyError('Unresolvable Reference: Probable circular initialization'+

ipdb> model.components
*** NameError: name 'model' is not defined
ipdb> ls
*** NameError: name 'ls' is not defined
ipdb> w
  <ipython-input-2-7798442bc851>(1)<module>()
----> 1 pysd.read_vensim('./tests/test-models/tests/subscript_3d_arrays/test_subscript_3d_arrays.mdl')

  /Users/houghton/Google_Drive/Academic Projects/PYSD/pysd/pysd/pysd.py(62)read_vensim()
     60     from vensim2py import translate_vensim
     61     py_model_file = translate_vensim(mdl_file)
---> 62     model = load(py_model_file)
     63     return model
     64 

  /Users/houghton/Goo

In [47]:
np.ones(3)*NaN

array([ nan,  nan,  nan])

In [2]:
import xarray as xr

In [4]:
coords = {'Second Dimension Subscript': ['Column 1', 'Column 2'],
          'Third Dimension Subscript': ['Depth 1', 'Depth 2'],
          'One Dimensional Subscript': ['Entry 1', 'Entry 2', 'Entry 3']}
coords

{'One Dimensional Subscript': ['Entry 1', 'Entry 2', 'Entry 3'],
 'Second Dimension Subscript': ['Column 1', 'Column 2'],
 'Third Dimension Subscript': ['Depth 1', 'Depth 2']}

In [43]:
a = xr.DataArray(data=np.zeros(map(len, coords.values()))*NaN, coords=coords)

In [72]:
m = np.empty([3])*NaN
m

array([ nan,  nan,  nan])

In [70]:
np.empty(4)

array([ 1.,  2.,  3.,  4.])

In [49]:
a.sel(One Dimensional Subscript='Column 1')

SyntaxError: invalid syntax (<ipython-input-49-f3f8947e1e9d>, line 1)

In [36]:
a.loc[{'One Dimensional Subscript': ['Entry 1']}]

<xarray.DataArray (Second Dimension Subscript: 2, Third Dimension Subscript: 2, One Dimensional Subscript: 1)>
array([[[ 1.],
        [ 2.]],

       [[ 3.],
        [ 4.]]])
Coordinates:
  * Second Dimension Subscript  (Second Dimension Subscript) |S8 'Column 1' ...
  * Third Dimension Subscript   (Third Dimension Subscript) |S7 'Depth 1' ...
  * One Dimensional Subscript   (One Dimensional Subscript) |S7 'Entry 1'

In [ ]:
a.loc[{'One Dimensional Subscript': ['Entry 1']}]

In [18]:
a.loc[:,:,'Entry 1']

<xarray.DataArray (Second Dimension Subscript: 2, Third Dimension Subscript: 2)>
array([[ 1.,  2.],
       [ 0.,  0.]])
Coordinates:
  * Second Dimension Subscript  (Second Dimension Subscript) |S8 'Column 1' ...
  * Third Dimension Subscript   (Third Dimension Subscript) |S7 'Depth 1' ...
    One Dimensional Subscript   |S7 'Entry 1'

In [37]:
a.loc[:,'Entry 1',:]

KeyError: 'Entry 1'

In [16]:
a.loc['Column 1',:,'Entry 1']

<xarray.DataArray (Third Dimension Subscript: 2)>
array([ 0.,  0.])
Coordinates:
    Second Dimension Subscript  |S8 'Column 1'
  * Third Dimension Subscript   (Third Dimension Subscript) |S7 'Depth 1' ...
    One Dimensional Subscript   |S7 'Entry 1'

In [17]:
a.loc['Column 1',:,'Entry 1'] = 1, 2

In [21]:
a.loc[:,:,'Entry 1'] = [1,2],[3,4]

In [22]:
a

<xarray.DataArray (Second Dimension Subscript: 2, Third Dimension Subscript: 2, One Dimensional Subscript: 3)>
array([[[ 1.,  0.,  0.],
        [ 2.,  0.,  0.]],

       [[ 3.,  0.,  0.],
        [ 4.,  0.,  0.]]])
Coordinates:
  * Second Dimension Subscript  (Second Dimension Subscript) |S8 'Column 1' ...
  * Third Dimension Subscript   (Third Dimension Subscript) |S7 'Depth 1' ...
  * One Dimensional Subscript   (One Dimensional Subscript) |S7 'Entry 1' ...

In [25]:
a.loc[:,:,'Entry 1'] = [1,2],[3,4]

In [26]:
a

<xarray.DataArray (Second Dimension Subscript: 2, Third Dimension Subscript: 2, One Dimensional Subscript: 3)>
array([[[ 1.,  0.,  0.],
        [ 2.,  0.,  0.]],

       [[ 3.,  0.,  0.],
        [ 4.,  0.,  0.]]])
Coordinates:
  * Second Dimension Subscript  (Second Dimension Subscript) |S8 'Column 1' ...
  * Third Dimension Subscript   (Third Dimension Subscript) |S7 'Depth 1' ...
  * One Dimensional Subscript   (One Dimensional Subscript) |S7 'Entry 1' ...

In [50]:
m = {'Dim1': ['A', 'B'],
                                      'Dim2': ['C', 'D', 'E'],
                                      'Dim3': ['F', 'G', 'H', 'I']}

In [53]:
m.iterkeys()

<dictionary-keyiterator at 0x10d7459f0>

In [56]:
a = 'hi'
a = a or None
a

'hi'

In [57]:
b = None
b = b or 'hi'
b

'hi'

In [60]:
c = 'hi there'
c = c or b
c

'hi there'

In [61]:
b = None
b = b + 'hi'
b

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [62]:
a = ['hi']
a += ['bob']
a

['hi', 'bob']

In [66]:
 or 'hi'

' '

In [64]:
None + 'hi'  'there'

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [73]:
import autopep8

In [82]:
print autopep8.fix_code("xr.DataArray(data=np.empty([3])*NaN, coords={'Dim1': ['A','B','C'], 'Dim1': ['A', 'B', 'C'], 'Dim1': ['A', 'B', 'C'] 'Dim2': ['D', 'E', 'F']})')", options={'aggressive': 1})

xr.DataArray(data=np.empty([3]) * NaN, coords={'Dim1': ['A', 'B', 'C'], 'Dim1': ['A', 'B', 'C'], 'Dim1': ['A', 'B', 'C'] 'Dim2': ['D', 'E', 'F']})')



In [79]:
len("xr.DataArray(data=np.empty([3])*NaN, coords={'Dim1': ['A', 'B', 'C'], 'Dim1': ['A', 'B', 'C'], 'Dim1': ['A', 'B', 'C'] 'Dim2': ['D', 'E', 'F']})'))")

147

In [84]:
repr(map(len, coords.values()))

'[2, 2, 3]'

In [85]:
import textwrap

In [ ]:
textwrap.

In [86]:
string = """hi
this
is 
my
string"""


In [4]:
from numpy import testing as

In [5]:
import numpy.testing as npt

In [6]:
import pandas.tests as pdt

In [8]:
import pandas.util.testing as pdt

In [10]:
dir(pdt)

['CategoricalIndex',
 'Counter',
 'DataFrame',
 'DatetimeIndex',
 'Index',
 'K',
 'LooseVersion',
 'MultiIndex',
 'N',
 'Panel',
 'Panel4D',
 'PeriodIndex',
 'RANDS_CHARS',
 'RANDU_CHARS',
 'RNGContext',
 'RangeIndex',
 'Series',
 'SimpleMock',
 'SubclassedDataFrame',
 'TestCase',
 'TestSubDict',
 'TimedeltaIndex',
 '_AssertRaisesContextmanager',
 '_RAISE_NETWORK_ERROR_DEFAULT',
 '__builtins__',
 '__doc__',
 '__file__',
 '__name__',
 '__package__',
 '_can_set_locale',
 '_create_missing_idx',
 '_default_locale_getter',
 '_incompat_bottleneck_version',
 '_network_errno_vals',
 '_network_error_classes',
 '_network_error_messages',
 '_panel_frame_equal',
 '_skip_if_32bit',
 '_skip_if_has_locale',
 '_skip_if_mpl_1_5',
 '_skip_if_no_dateutil',
 '_skip_if_no_localpath',
 '_skip_if_no_mpl',
 '_skip_if_no_pathlib',
 '_skip_if_no_pytz',
 '_skip_if_no_scipy',
 '_skip_if_no_xarray',
 '_skip_if_scipy_0_17',
 '_skip_if_windows',
 '_skip_if_windows_python_3',
 '_testing',
 '_valid_locales',
 'add_nan

In [3]:
import imp

In [5]:
test = imp.load_source('components', 'tests/test-models/samples/teacup/teacup.py')

In [7]:
test2 = imp.load_source('components', 'tests/test-models/samples/SIR/SIR.py')

In [9]:
test3 = imp.load_source('different', 'tests/test-models/samples/SIR/SIR.py')

In [1]:
from pysd.functions import cache

/Users/houghton/anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [6]:
test.

<module 'components' from 'tests/test-models/samples/teacup/teacup.pyc'>

In [8]:
dir(test2)

['__builtins__',
 '__doc__',
 '__file__',
 '__name__',
 '__package__',
 '_dinfectious_dt',
 '_drecovered_dt',
 '_dsusceptible_dt',
 '_dteacup_temperature_dt',
 '_init_infectious',
 '_init_recovered',
 '_init_susceptible',
 '_init_teacup_temperature',
 'cache',
 'characteristic_time',
 'contact_infectivity',
 'division',
 'duration',
 'final_time',
 'functions',
 'heat_loss_to_room',
 'infectious',
 'initial_time',
 'namespace',
 'np',
 'recovered',
 'recovering',
 'room_temperature',
 'saveper',
 'subscript_dict',
 'succumbing',
 'susceptible',
 'teacup_temperature',
 'time_step',
 'total_population']

In [10]:
dir(test3)

['__builtins__',
 '__doc__',
 '__file__',
 '__name__',
 '__package__',
 '_dinfectious_dt',
 '_drecovered_dt',
 '_dsusceptible_dt',
 '_init_infectious',
 '_init_recovered',
 '_init_susceptible',
 'cache',
 'contact_infectivity',
 'division',
 'duration',
 'final_time',
 'functions',
 'infectious',
 'initial_time',
 'namespace',
 'np',
 'recovered',
 'recovering',
 'saveper',
 'subscript_dict',
 'succumbing',
 'susceptible',
 'time_step',
 'total_population']

In [11]:
import time

In [12]:
time.perf_counter()

AttributeError: 'module' object has no attribute 'perf_counter'

In [13]:
import os


'\xddq\x9cn\x97'

In [31]:
%timeit os.urandom(5)

The slowest run took 6.50 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.43 µs per loop


In [17]:
%timeit str(time.time()).replace('.','')

The slowest run took 9.76 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 611 ns per loop


In [28]:
import random
import string

In [22]:
%timeit str(random.randint(0,1000000))

The slowest run took 7.60 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.19 µs per loop


In [30]:
%timeit ''.join([random.choice(string.lowercase) for _ in range(10)])

100000 loops, best of 3: 6.7 µs per loop


In [ ]:
%timeit random.

In [16]:
%timeit os.urandom(5)

The slowest run took 48.35 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.39 µs per loop
